In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[all]

In [ ]:
import ee
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import geopandas
import os
import requests

In [ ]:
## specify areas of interest / districts
## URL method accesses an UrbanShift city's boundaries and uses information from file name and geoBoundaries properties ("shapeName") to create properties for output file
#URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM1/boundary-CHN-Ningbo-ADM1.geojson'
URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/cities4forests/data/boundaries/MEX-Mexico_City-ADM2.geojson'
DistrictsGJ = requests.get(URL).json()
Districts = geemap.geojson_to_ee(DistrictsGJ)
#Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
DistrictsProjCRS = Districts.geometry().projection().crs()

print(DistrictsProjCRS.getInfo())
print(Districts.first().toDictionary().getInfo())

In [ ]:
# extract area properties from standarized filename
# https://note.nkmk.me/en/python-split-rsplit-splitlines-re/ 
basename = os.path.splitext(os.path.basename(URL))[0]
AOIname = basename.split('-',1)[1].rsplit('-',1)[0]
#ADMlevel = basename.split('-')[-1]

Areaofinterest = AOIname ## 3-letter country abreviation - city name with underscore for spaces, e.g. "ETH-Addis_Ababa"
#unitofanalysis = ADMlevel ## options: "ADM0" (country), "ADM1" (state), "Metro" (metropolitan region), "ADM2" (municipality), "ADM3" (subcity/locality), "ADM4"(ward/neighborhood), ideally align with https://www.geoboundaries.org/index.html#getdata

print(Areaofinterest)
#print(unitofanalysis)

In [ ]:
## create map
Map = geemap.Map(height="350px")

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(Districts, zoom=10)

In [ ]:
Map.addLayer(Districts,{},"Districts")
#Map

In [ ]:
pop = ee.ImageCollection('WorldPop/GP/100m/pop_age_sex_cons_unadj')
pop = (pop.filter(ee.Filter.And(
    ee.Filter.bounds(Districts),
    ee.Filter.inList('year',[2020])))
    .select('population'))
popImg = pop.mean().rename('population')

popProj = pop.first().projection()
popScale = popProj.nominalScale()
popProj10m = popProj.atScale(10)
popScale10m = popProj10m.nominalScale()

popviz = {
  'min': 0.0,
  'max': 150.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
};
       
Map.addLayer(popImg,popviz,"Population")

In [ ]:
# get bounding box for Districts area

bb = Districts.union(1).geometry().buffer(100).bounds(1000)
print(bb.getInfo())
#Map.addLayer(bb,{},"Bounding box")

# get N, S, E, W coordinates from bounding box
# from https://gis.stackexchange.com/questions/318959/get-lon-lat-of-a-top-left-corner-for-geometry-in-google-earth-engine

## return the list of coordinates
listCoords = ee.Array.cat(bb.coordinates(), 1); 
##Casting it to an array makes it possible to slice out the x and y coordinates:
##get the X-coordinates
xCoords = listCoords.slice(1, 0, 1); #print('xCoords', xCoords.getInfo());
yCoords = listCoords.slice(1, 1, 2); #print('yCoords', yCoords.getInfo());

## Reducing each array reveals then the min and max values:
## reduce the arrays to find the max (or min) value
West = ee.Number(xCoords.reduce(ee.Reducer.min(), [0]).get([0,0])).getInfo(); #print('West',West);
East = ee.Number(xCoords.reduce(ee.Reducer.max(), [0]).get([0,0])).getInfo(); #print('East',East);
North = ee.Number(yCoords.reduce(ee.Reducer.min(), [0]).get([0,0])).getInfo(); #print('North',North);
South = ee.Number(yCoords.reduce(ee.Reducer.max(), [0]).get([0,0])).getInfo(); #print('South',South);

In [ ]:
# use bounding box to get geodataframe of all OSM data on recreation sites/parks. Use .drop to remove properties that have incompatible names with FeatureCollections

tags = {'leisure':['park','nature_reserve','common','playground','pitch','track'],'boundary':['protected_area','national_park']} 
RecSites = geemap.osm_gdf_from_bbox(North, South, East, West, tags)#.drop(columns=['contact:p.o.box']) # use to remove properties that have incompatible names with FeatureCollections
RecSites = RecSites.reset_index()
print(RecSites.shape[0])
print(RecSites.crs)
RecSites.sort_index()#.sample(3)
#RecSites.sort_values(by=['osmid'],axis='index')

In [ ]:
## keep only columns desired to reduce file size 
RecSites = RecSites[RecSites['element_type']!= 'node']
RecSites = RecSites[RecSites.geom_type != 'LineString']
RecSites = RecSites.loc[:, ['osmid','geometry']] #['element_type','osmid','leisure','boundary','name','geometry']
print(RecSites.shape[0])
RecSites.sort_index()#.sample(3)

In [ ]:
# convert Geodataframe to GeoJson
RecSitesGJ = geemap.gdf_to_geojson(RecSites) #RecSitesSimp
# convert GeoJson to ee.FeatureCollection
RecSitesFC = geemap.geojson_to_ee(RecSitesGJ)#.filter(ee.Filter.eq('element_type','way')).select(['osmid','element_type','geometry','leisure','boundary'])

In [ ]:
# add recreation sites to map 
Map.addLayer(RecSitesFC, {}, 'Recreation Sites')

In [ ]:
DistanceThres = 400 # meters distance from population to be considered

def amenityBuffer(feat):
  feat = ee.Feature(feat)
  return feat.buffer(DistanceThres)
RecCatchment = RecSitesFC.map(amenityBuffer)

RecCatchmentUnion = RecCatchment.union()
Map.addLayer(RecCatchmentUnion, {}, 'Recreation catchment')


In [ ]:
mask = ee.Image.constant(1).clip(RecCatchmentUnion.geometry()).mask()

popwOSaccess = popImg.updateMask(mask).rename('populationwOpenSpace')
#Map.addLayer(popwOSaccess, popviz, 'Population with open space access')

Map

In [ ]:
## add tree cover dataset
TML = ee.ImageCollection('projects/wri-datalab/TML')
TreeCover = TML.reduce(ee.Reducer.mean()).rename('b1')
#TreeCover = TreeCover.resample('bilinear').reproject(popProj10m)

Map2 = geemap.Map(height="350px")
Map2.add_basemap('HYBRID')
Map2.centerObject(Districts, zoom=10)

Map2.addLayer(popImg,popviz,"Population",False)

TreePctThreshold = 10 #whole numbers - 0-100, minimum percentage of tree cover threshold to consider 

Map2.addLayer(TreeCover.updateMask(TreeCover.gte(TreePctThreshold)),
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Tree Cover 2020 (WRI Trees in Mosaic Landscapes)',True,1)

In [ ]:
# calcs for % population with threshold level (e.g. 10%+) of tree cover within walking distance (e.g. 400m)

circleTheshm = ee.Kernel.circle(DistanceThres, 'meters', False)
TreeCoverinThreshm = TreeCover.reduceNeighborhood(ee.Reducer.mean(), circleTheshm)

popwthresTC = popImg.updateMask(TreeCoverinThreshm.gte(TreePctThreshold)).rename('populationwTreeCover')

Map2.addLayer(TreeCoverinThreshm,{'min':0, 'max':100},'TreeCoverin400m',False)
Map2.addLayer(popwthresTC, popviz, 'Population with access to tree cover')
Map2

In [ ]:
combImg = popImg.addBands([popwOSaccess,popwthresTC])

PopbyDistrict=combImg.reduceRegions(
  reducer= ee.Reducer.sum(), 
  collection= Districts, 
  scale= popScale, 
  tileScale= 1
)

#print(PopbyDistrict.first().toDictionary().getInfo())

In [ ]:
def calcs(feat):
     return feat.set({
        'PopwOpenSpaceAccessPct': feat.getNumber('populationwOpenSpace').divide(feat.getNumber('population')),
        'PopwTreeCoverAccessPct': feat.getNumber('populationwTreeCover').divide(feat.getNumber('population'))
     })
         
PopbyDistrict = PopbyDistrict.map(calcs).sort('PopwTreeCoverAccessPct',False)
#print(PopbyDistrict.first().toDictionary().getInfo())

In [ ]:
## display features in chart

import geemap.chart as chart

xProperty = 'shapeName' #'geo_name' #,"Woreda"
yProperties = ['PopwTreeCoverAccessPct'] # ,'LC50areaKM2'

options = {
    'xlabel': "District",
    'ylabel': "% population with at least 10% tree cover within 5 minute walking distance",
    "legend_location": "top-right",
    "height": "500px",
}

chart.feature_byFeature(PopbyDistrict, xProperty, yProperties, **options)

In [ ]:
# Download Image to Google Drive

# Set configuration parameters for output vector
task_config = {
    #'folder': 'gee-data',  # output Google Drive folder
    'region':Districts.geometry(),
    'scale':10
}
#print('Exporting {}'.format(OpenBuiltAreaPct))
#task = ee.batch.Export.image.toDrive(combImg, 'OS+TCimage', **task_config)
#task.start()